In [ ]:
https://drive.google.com/file/d/1vcpArYl_qzLsK_4t9GRrh83UxWcc-vOH/view?usp=sharing ppnp
https://drive.google.com/file/d/1TcOhjWZlqgQCJlGpZAjlo-mni4N50FF0/view?usp=sharing mine
https://drive.google.com/file/d/1vqVbt_ifOg5-igKSrRFHfgXOzwdXZvyf/view?usp=sharing att
https://drive.google.com/file/d/1jSVmOK6OqkqUoxb9fXt516Pl0cey2pw4/view?usp=sharing hgrn
https://drive.google.com/file/d/1CERSDfUJFRUFoKHXHBgu97Lh-07GUTOK/view?usp=sharing hgrn_pro

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive
! pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth=GoogleAuth()
gauth.credentials=creds
drive=GoogleDrive(gauth)

In [3]:
download=drive.CreateFile({'id':'1jSVmOK6OqkqUoxb9fXt516Pl0cey2pw4'})
download.GetContentFile('hgrn-master.zip')

! unzip hgrn-master.zip



Archive:  hgrn-master.zip
   creating: hgrn-master/
  inflating: hgrn-master/.gitignore  
   creating: hgrn-master/.idea/
  inflating: hgrn-master/.idea/.gitignore  
   creating: hgrn-master/.idea/inspectionProfiles/
  inflating: hgrn-master/.idea/inspectionProfiles/profiles_settings.xml  
  inflating: hgrn-master/.idea/misc.xml  
  inflating: hgrn-master/.idea/modules.xml  
  inflating: hgrn-master/.idea/ppnp-master.iml  
  inflating: hgrn-master/.idea/workspace.xml  
   creating: hgrn-master/.ipynb_checkpoints/
  inflating: hgrn-master/.ipynb_checkpoints/HGRN-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoints/ppnp (3)-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoints/ppnp-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoints/ppnp_citeseer-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoints/ppnp_cora-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoints/reproduce_results-checkpoint.ipynb  
  inflating: hgrn-master/.ipynb_checkpoint

In [1]:
%cd hgrn-master/
import logging
import argparse
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from hgrn.hgrn import HGRN
from hgrn.hgrn.training import train_model
from hgrn.hgrn.earlystopping import stopping_args
from hgrn.hgrn.highOrderInfo import InfoPropagation
from hgrn.data.io import load_dataset

/content/hgrn-master
Instructions for updating:
non-resource variables are not supported in the long term
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [2]:
tf.logging.set_verbosity(tf.logging.WARN)
logging.basicConfig(
        format='%(asctime)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO + 2)

In [3]:
graph_name = 'cora_ml'
graph = load_dataset(graph_name)
graph.standardize(select_lcc=True)

<Undirected, unweighted and connected SparseGraph with 15962 edges (no self-loops). Data: adj_matrix (2810x2810), attr_matrix (2810x2879), labels (2810), node_names (2810), attr_names (2879), class_names (7)>

In [4]:
test = True
test_seeds = [
        2144199730,  794209841, 2985733717, 2282690970, 1901557222,
        2009332812, 2266730407,  635625077, 3538425002,  960893189,
        497096336, 3940842554, 3594628340,  948012117, 3305901371,
        3644534211, 2297033685, 4092258879, 2590091101, 1694925034]
val_seeds = [
        2413340114, 3258769933, 1789234713, 2222151463, 2813247115,
        1920426428, 4272044734, 2092442742, 841404887, 2188879532,
        646784207, 1633698412, 2256863076,  374355442,  289680769,
        4281139389, 4263036964,  900418539,  119332950, 1628837138]

if test:
    seeds = test_seeds
else:
    seeds = val_seeds

In [5]:
if graph_name == 'ms_academic':
    nknown = 5000
else:
    nknown = 1500
    
idx_split_args = {'ntrain_per_class': 20, 'nstopping': 500, 'nknown': nknown}

In [6]:
prop_hgrn = InfoPropagation(niter=15)


In [7]:
"""
model_args = {
    'hiddenunits': [64],
    'reg_lambda': 5e-3,
    'lambda_graph': 0.2
    'learning_rate': 0.01,
    'keep_prob': 0.5,
    'keep_feature':0.5,
    'keep_prob_layers': 1,
    'propagation': prop_hgrn}

"""
model_args = {
    'hiddenunits': [64],
    'reg_lambda': 5e-3,
    'lambda_graph': 4e-3, 
    'learning_rate': 0.01,
    'keep_prob': 0.5,
    'keep_feature': 1,
    'keep_prob_layers': 0.8,
    'propagation': prop_hgrn}

In [8]:
niter_per_seed = 5
save_result = False
print_interval = 100


In [9]:
results = []
niter_tot = niter_per_seed * len(seeds)
i_tot = 0
for seed in seeds:
    idx_split_args['seed'] = seed
    for _ in range(niter_per_seed):
        i_tot += 1
        logging_string = f"Iteration {i_tot} of {niter_tot}"
        logging.log(22,
                logging_string + "\n                     "
                + '-' * len(logging_string))
        result = train_model(
                graph_name, HGRN, graph, model_args, idx_split_args,
                stopping_args, test, save_result, None, print_interval)
        results.append({})
        results[-1]['stopping_accuracy'] = result['early_stopping']['accuracy']
        results[-1]['stopping_f1_score'] = result['early_stopping']['f1_score']
        results[-1]['valtest_accuracy'] = result['valtest']['accuracy']
        results[-1]['valtest_f1_score'] = result['valtest']['f1_score']
        results[-1]['runtime'] = result['runtime']
        results[-1]['runtime_perepoch'] = result['runtime_perepoch']
        results[-1]['split_seed'] = seed

2021-07-05 05:47:35: Iteration 1 of 100
                     ------------------
2021-07-05 05:47:35: Tensorflow seed: 2047054690


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2021-07-05 05:47:36: From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:206: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2021-07-05 05:47:55: Last step: 1079, best step: 726 (18.647 sec)
2021-07-05 05:47:55: Test accuracy: 83.6%, test F1 score: 0.836
2021-07-05 05:47:55: Iteration 2 of 100
                     ------------------
2021-07-05 05:47:55: Tensorflow seed: 1466679115
2021-07-05 05:48:17: Last step: 1263, best step: 895 (21.219 sec)
2021-07-05 05:48:17: Test accuracy: 83.7%, test F1 score: 0.834
2021-07-05 05:48:17: Iteration 3 of 100
                     ------------------
2021-07-05 05:48:17: Tensorflow seed: 3449992916
2021-07-05 05:48:40: Last step: 1339, best step: 706 (22.439 sec)
2021-07-05 05:48:40: Test accuracy: 83.7%, test F1 score: 0.838
2021-07-05

In [10]:
import pandas as pd
import seaborn as sns
result_df = pd.DataFrame(results)
result_df.head()

,stopping_accuracy,stopping_f1_score,valtest_accuracy,valtest_f1_score,runtime,runtime_perepoch,split_seed
0,0.820,0.812399,0.835878,0.836368,18.647210,0.017266,2144199730
1,0.832,0.818576,0.837405,0.833853,21.218623,0.016787,2144199730
2,0.830,0.825871,0.837405,0.837585,22.439228,0.016746,2144199730
3,0.844,0.830422,0.858779,0.854347,35.982539,0.016838,2144199730
4,0.838,0.818885,0.849618,0.842593,37.246095,0.016876,2144199730


In [11]:
 def calc_uncertainty(values: np.ndarray, n_boot: int = 1000, ci: int = 95) -> dict:
    stats = {}
    stats['mean'] = values.mean()
    boots_series = sns.algorithms.bootstrap(values, func=np.mean, n_boot=n_boot)
    stats['CI'] = sns.utils.ci(boots_series, ci)
    stats['uncertainty'] = np.max(np.abs(stats['CI'] - stats['mean']))
    return stats

In [12]:
stopping_acc = calc_uncertainty(result_df['stopping_accuracy'])
stopping_f1 = calc_uncertainty(result_df['stopping_f1_score'])
valtest_acc = calc_uncertainty(result_df['valtest_accuracy'])
valtest_f1 = calc_uncertainty(result_df['valtest_f1_score'])
runtime = calc_uncertainty(result_df['runtime'])
runtime_perepoch = calc_uncertainty(result_df['runtime_perepoch'])

In [13]:
print("HGRN\n"
      "Early stopping: Accuracy: {:.2f} ± {:.2f}%, "
      "F1 score: {:.4f} ± {:.4f}\n"
      "{}: Accuracy: {:.2f} ± {:.2f}%, "
      "F1 score: {:.4f} ± {:.4f}\n"
      "Runtime: {:.3f} ± {:.3f} sec, per epoch: {:.2f} ± {:.2f}ms"
      .format(
          stopping_acc['mean'] * 100,
          stopping_acc['uncertainty'] * 100,
          stopping_f1['mean'],
          stopping_f1['uncertainty'],
          'Test' if test else 'Validation',
          valtest_acc['mean'] * 100,
          valtest_acc['uncertainty'] * 100,
          valtest_f1['mean'],
          valtest_f1['uncertainty'],
          runtime['mean'],
          runtime['uncertainty'],
          runtime_perepoch['mean'] * 1e3,
          runtime_perepoch['uncertainty'] * 1e3,
      ))

HGRN
Early stopping: Accuracy: 84.76 ± 0.33%, F1 score: 0.8328 ± 0.0042
Test: Accuracy: 85.79 ± 0.21%, F1 score: 0.8492 ± 0.0018
Runtime: 31.629 ± 1.694 sec, per epoch: 16.99 ± 0.04ms
